Задание 1.
Обязательная часть
Будем парсить страницу со свежеми новостям на habr.com/ru/all/.
Вам необходимо собирать только те статьи, в которых встречается хотя бы одно требуемое ключевое слово. Эти слова определяем в начале кода в переменной, например:
KEYWORDS = ['python', 'парсинг']
Поиск вести по всей доступной preview-информации (это информация, доступная непосредственно с текущей страницы).
В итоге должен формироваться датафрейм вида: <дата> - <заголовок> - <ссылка>
Дополнительная часть (необязательная)
Улучшить скрипт так, чтобы он анализировал не только preview-информацию статьи, но и весь текст статьи целиком.
Для этого потребуется получать страницы статей и искать по тексту внутри этой страницы.
Итоговый датафрейм формировать со столбцами: <дата> - <заголовок> - <ссылка> - <текст_статьи>


In [68]:
import requests

In [69]:
import time
time.sleep(0.2)

In [70]:
from bs4 import BeautifulSoup

In [71]:
import pandas as pd

In [72]:
#KEYWORDS = ['python', 'парсинг'] #Ничего не находит, найдем другие слова
KEYWORDS = ['контекст', 'статья'] 

In [73]:
req = requests.get('https://habr.com/ru/all/')
soup = BeautifulSoup(req.text, 'html.parser')

In [74]:
posts = soup.find_all('article', class_='post_preview')
# len(posts)
# posts

In [77]:
news_list = []
for post in posts:
    post_id = post.parent.attrs.get('id')
   
    if not post_id:
        continue
    
    post_id = int(post_id.split('_')[-1])
    text_preview = post.find_all('div', class_='post__text')
    
    for preview in text_preview:
#         print(preview)
        preview_lower = preview.text.lower()
#         print(preview_lower) 
        if any([desired in preview_lower for desired in KEYWORDS]):
#             print(preview_lower)
                
            date_element = post.find('span', class_ = 'post__time')
            title_element = post.find('a', class_='post__title_link')
            link_element = title_element.attrs.get('href')
           
            news_list.append({'date': date_element.text, 'title': title_element.text, 'link': link_element})
            
df = pd.DataFrame(news_list)
df

,date,title,link
0,сегодня в 19:00,Подключение автоплатежей через TeleWalletAbot ...,https://habr.com/ru/post/524316/
1,сегодня в 18:02,Онлайн-встреча по информационной безопасности ...,https://habr.com/ru/company/dsec/blog/524338/


Задание 2.
Обязательная часть
Написать скрипт, который будет проверять список e-mail адресов на утечку при помощи сервиса Avast Hack Ckeck. Список email-ов задаем переменной в начале кода:
EMAIL = [xxx@x.ru, yyy@y.com]
В итоге должен формироваться датафрейм со столбцами: <почта> - <да
Подсказка: сервис работает при помощи "скрытого" API. Внимательно изучите post-запросы.
Дополнительная часть (необязательная)
Написать скрипт, который будет получать 50 последних постов указанной группы во Вконтакте.
Документация к API VK: https://vk.com/dev/methods , вам поможет метод wall.get
GROUP = 'netology'  
TOKEN = УДАЛЯЙТЕ В ВЕРСИИ ДЛЯ ПРОВЕРКИ, НА GITHUB НЕ ВЫКЛАДЫВАТЬ
В итоге должен формироваться датафрейм со столбцами: <дата поста> - <текст поста>


In [84]:
import json
import datetime
EMAIL = ['example@gmail.com', 'example@mail.ru']
list_ = []

In [86]:
for i in range(0, len(EMAIL)):
    data_email = {'email': EMAIL[i]}
    data_email_json = json.dumps(data_email)
    req = requests.post('https://digibody.avast.com/v1/web/leaks', data = data_email_json)
    req_dict = req.json()
#     g = req_dict['value'][i]['description']
#     print(req_dict)
#     print(g)
    
    for i in range(0, len(req_dict['value'])):
    
        email = req_dict['value'][i]['username']
         
        leak_date = req_dict['value'][i]['leak_date']
        dt = datetime.datetime.fromtimestamp(leak_date / 1000)
        formatted_time = dt.isoformat(sep=' ', timespec='milliseconds')
        
        leak_source = req_dict['value'][i]['domain']
        
    #     leak_description = req_dict['value'][i]['description']
    
    #     list_.append({'почта': email, 'дата утечки': leak_date, 'источник утечки': leak_source, 'описание утечки': leak_description})
        list_.append({'почта': email, 'дата утечки': formatted_time, 'источник утечки': leak_source})

data = pd.DataFrame(list_)
data

,почта,дата утечки,источник утечки
0,None,2019-07-25 03:00:00.000,None
1,None,2020-07-23 03:00:00.000,None
2,None,2019-04-04 03:00:00.000,None
3,None,2019-10-17 03:00:00.000,None
4,None,2019-07-25 03:00:00.000,None
...,...,...,...
230,None,2020-07-23 03:00:00.000,None
231,None,2020-02-13 03:00:00.000,None
232,None,2020-09-24 03:00:00.000,None
233,None,2019-12-05 03:00:00.000,None
